## Initial Data Cleaning

- Fix convert the datetime column to datetime type
- Add a column for date of week of checkout
- Add a column for the different bins, we defined for the checkout time
  -  0: 00:00AM - 5:00AM
  -  1: 5AM-10AM
  -  2: 10AM-2PM
  -  3: 2PM-6PM
  -  4: 6PM-10:30PM
  -  5: 10:30PM-00:00AM


In [26]:
import datetime
import os
import dotenv
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)

dotenv.load_dotenv()

True

In [27]:
bike_trips = os.path.join(os.environ['MOHAMMAD_SHARED_PATH'], 'Backup', 'Bike_Trips.xlsx')

In [28]:
df_bike_share = pd.read_excel(bike_trips, sheet_name="Data_Reformatted")   
#### Remove Duration minute == 0
df_bike_share = df_bike_share[df_bike_share['Duration (Minutes)'] != 0].copy()

df_bike_share = df_bike_share.fillna(0)

#### 2 of the rows in the df where misaligned

In [29]:
# Filter out only datetime.time objects
time_mask_ok = df_bike_share['Checkout Time'].apply(lambda x: isinstance(x, datetime.time))
time_df_ok = df_bike_share[time_mask_ok].copy()

time_mask_notOk = df_bike_share['Checkout Time'].apply(lambda x: not isinstance(x, datetime.time))
time_df_notOk = df_bike_share[time_mask_notOk].copy()
time_df_notOk = time_df_notOk.shift(axis=1)

df_bike_share = pd.concat([time_df_ok, time_df_notOk], axis=0)

#there is no some return date time
mask = df_bike_share['Return Time'].apply(lambda x: isinstance(x, datetime.time))
df_bike_share = df_bike_share[mask].copy()

#### Convert the date and time column to a single datetime column of 
`type(object), type(object)` -> `type(datetime)`

In [30]:
# Convert 'Checkout Date' to datetime
df_bike_share['Checkout Date'] = pd.to_datetime(df_bike_share['Checkout Date'])
df_bike_share['Return Date'] = pd.to_datetime(df_bike_share['Return Date']).copy()

# Extract time component from 'Checkout Time'
time_component_checkout = df_bike_share['Checkout Time'].apply(lambda x: x.strftime('%H:%M:%S'))

time_component_return = df_bike_share['Return Time'].apply(lambda x: x.strftime('%H:%M:%S'))

# Combine date from 'Checkout Date' and time from 'Checkout Time'
df_bike_share['Checkout DateTime'] = df_bike_share['Checkout Date'] + pd.to_timedelta(time_component_checkout)
df_bike_share['Return DateTime'] = df_bike_share['Return Date'] + pd.to_timedelta(time_component_return)

df_bike_share = df_bike_share.drop(columns=['Checkout Date', 'Checkout Time', 'Return Date', 'Return Time'])

### Add day of the week
df_bike_share['Day_of_week'] = df_bike_share['Checkout DateTime'].dt.day_of_week

##### Breakdown the checkout time into bins

In [31]:
# 0: 00:00AM - 5:00AM
# 1: 5AM-10AM
# 2: 10AM-2PM
# 3: 2PM-6PM
# 4: 6PM-10:30PM
# 5: 10:30PM-00:00AM



# Define the bins and labels for time frames
bins = [-1, 300, 600, 840, 1080, 1350, 1440]  # Bins covering the entire day in minutes
labels = [0, 1, 2, 3, 4, 5]  # Labels for each time frame

# Convert 'Checkout DateTime' to minutes since midnight
checkout_minutes = df_bike_share['Checkout DateTime'].dt.hour * 60 + df_bike_share['Checkout DateTime'].dt.minute

# Cut the 'Checkout DateTime' column into specified time frames
df_bike_share['checkout_time_bin'] = pd.cut(checkout_minutes, bins=bins, labels=labels)
df_bike_share.sort_values(by=['checkout_time_bin'], inplace=True)


In [32]:
needed_columns = ['Membership Type', 'Bike', 'Checkout DateTime', 'Checkout Kiosk', 'Return DateTime', 'Return Kiosk', 'Duration (Minutes)',  'Day_of_week', 'checkout_time_bin']

df_bike_share['Membership Type'] = df_bike_share['Membership Type'].astype(str)
df_bike_share['Bike'] = df_bike_share['Bike'].astype(str)
df_bike_share['Checkout Kiosk'] = df_bike_share['Checkout Kiosk'].astype(str)
df_bike_share['Return Kiosk'] = df_bike_share['Return Kiosk'].astype(str)
df_bike_share['Adjusted Duration (Minutes)'] = df_bike_share['Adjusted Duration (Minutes)'].astype(int)


In [33]:
df_bike_share['New Column'] = df_bike_share.apply(lambda row: row['Duration (Minutes)'] if row['Adjusted Duration (Minutes)'] == 0 else row['Adjusted Duration (Minutes)'], axis=1)

In [34]:
df_bike_share.drop(columns=['Adjusted Duration (Minutes)', 'Duration (Minutes)'], inplace=True)
df_bike_share.rename(columns={'New Column': 'Duration (Minutes)'}, inplace=True)

In [35]:
df_bike_share

,Username,First Name,Last Name,Membership Type,Bike,Checkout Kiosk,Return Kiosk,Charge,Adjusted Charge,Unnamed: 15,Unnamed: 16,Unnamed: 17,Checkout DateTime,Return DateTime,Day_of_week,checkout_time_bin,Duration (Minutes)
236031,ngreene894@yahoo.com,Nathanael,Greene,FUN! Pass,18086,33rd & California,40th & Farnam,0.0,0.0,0,Users,0,2021-04-12 00:11:00,2021-04-12 01:04:00,0,0,53
71605,mccandrew1212@icloud.com,Andrew,mccaughey,FUN! Pass,23481,Kiewit Luminarium,10th & Cass,5.0,0.0,0,Users,0,2023-05-21 00:19:00,2023-05-21 01:36:00,6,0,77
71606,kileejean.molczyk@gmail.com,kilee,molczyk,FUN! Pass,18125,Kiewit Luminarium,10th & Cass,5.0,0.0,0,Users,0,2023-05-21 00:18:00,2023-05-21 01:41:00,6,0,83
71607,vincentcosxhka@yahoo.com,Vincent,Coschka,FUN! Pass,25058,Kiewit Luminarium,Bob Kerrey Pedestrian Bridge,5.0,0.0,0,Users,0,2023-05-21 00:18:00,2023-05-21 02:14:00,6,0,116
71608,tangemanbt@gmail.com,Jayme,Spencer,FUN! Pass,19894,Kiewit Luminarium,11th & Jackson,0.0,0.0,0,Users,0,2023-05-21 00:17:00,2023-05-21 00:46:00,6,0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224776,dakotaantillon@gmail.com,Damian,Martinez,FUN! Pass,16634,45th & Emile,15th & Howard,0.0,0.0,0,Users,0,2021-05-25 23:18:00,2021-05-26 09:09:00,1,5,60
224777,0,KAW SER POH,HTOO,24-Hour Pass Kiosk,18437,Tom Hanafan Park,Tom Hanafan Park,0.0,0.0,0,Users,0,2021-05-25 23:03:00,2021-05-25 23:26:00,1,5,23
224778,0,KAW SER POH,HTOO,24-Hour Pass Kiosk,16493,Tom Hanafan Park,Tom Hanafan Park,0.0,0.0,0,Users,0,2021-05-25 23:02:00,2021-05-25 23:28:00,1,5,26
139078,malikwoods932@gmail.com,malik,woods,Heartland Monthly Pass,19960,1516 Cuming St,36th & Farnam,0.0,0.0,0,Users,0,2022-06-12 23:28:00,2022-06-12 23:42:00,6,5,14


In [36]:
df_bike_share[needed_columns].to_parquet(os.path.join(os.environ['OUTPUT_PATH'], 'Cleaned_Bike_Trips.parquet'))